In [0]:
import pyspark.sql.functions as F
import os

STORAGE_ACCOUNT = os.getenv('STORAGE_ACCOUNT')
STORAGE_ACCOUNT_KEY = os.getenv('STORAGE_ACCOUNT_KEY')
spark.conf.set(STORAGE_ACCOUNT, STORAGE_ACCOUNT_KEY)


In [0]:
bangladesh_df = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("abfss://datasets@tfmstorageacc.dfs.core.windows.net/rape_total_2001_2021-bangladesh.csv")

bangladesh_df.limit(20).display()

Years,Total_victims,children,women,Unidentified_age
2021,1411,809,556,46
2020,1538,919,577,42
2019,1080,737,330,13
2018,635,457,176,2
2017,783,553,225,5
2016,757,511,232,14
2015,789,479,293,17
2014,666,393,244,29
2013,814,452,336,26
2012,805,473,299,33


This one is pretty clean already we just need a few adjustments

In [0]:
bangladesh_df_melted = bangladesh_df.drop(
    'Unidentified_age'
).melt(
    ids=['Years'],
    values=['Total_victims', 'children', 'women'],
    variableColumnName='age_group',
    valueColumnName='total_cases',
)
bangladesh_df_melted.limit(20).display()

Years,age_group,total_cases
2021,Total_victims,1411
2021,children,809
2021,women,556
2020,Total_victims,1538
2020,children,919
2020,women,577
2019,Total_victims,1080
2019,children,737
2019,women,330
2018,Total_victims,635


now change column names and add country information

In [0]:
age_group_dict = {
    'children':'child',
    'women':'adult',
    'Total_victims':'any'
}
bangladesh_df_melted.replace(
    age_group_dict,
    subset=['age_group'],
).withColumn(
    'country', F.lit('bangladesh'),
).select(
    F.col('Years').alias('year'),
    F.col('age_group'),
    F.col('total_cases'),
    F.col('country'),
).limit(20).display()

year,age_group,total_cases,country
2021,any,1411,bangladesh
2021,child,809,bangladesh
2021,adult,556,bangladesh
2020,any,1538,bangladesh
2020,child,919,bangladesh
2020,adult,577,bangladesh
2019,any,1080,bangladesh
2019,child,737,bangladesh
2019,adult,330,bangladesh
2018,any,635,bangladesh
